# 使用 OpenAI 工具进行提取

执行提取从未如此简单！OpenAI 的工具调用功能非常适合这个用途，因为它允许从文本中提取多个不同类型的不同元素。

1106 之后的模型使用工具并支持"并行函数调用"，这使得这一切变得超级简单。

In [8]:
from typing import List, Optional

from langchain.chains.openai_tools import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel
from langchain_openai import ChatOpenAI

In [2]:
# Make sure to use a recent model that supports tools
model = ChatOpenAI(model="gpt-3.5-turbo-1106")

In [3]:
# Pydantic is an easy way to define a schema
class Person(BaseModel):
    """Information about people to extract."""

    name: str
    age: Optional[int] = None

In [10]:
chain = create_extraction_chain_pydantic(Person, model)

In [11]:
chain.invoke({"input": "jane is 2 and bob is 3"})

[Person(name='jane', age=2), Person(name='bob', age=3)]

In [12]:
# Let's define another element
class Class(BaseModel):
    """Information about classes to extract."""

    teacher: str
    students: List[str]

In [13]:
chain = create_extraction_chain_pydantic([Person, Class], model)

In [14]:
chain.invoke({"input": "jane is 2 and bob is 3 and they are in Mrs Sampson's class"})

[Person(name='jane', age=2),
 Person(name='bob', age=3),
 Class(teacher='Mrs Sampson', students=['jane', 'bob'])]

## 深入原理

在底层，这是一个简单的链：

```python
from typing import Union, List, Type, Optional

from langchain.output_parsers.openai_tools import PydanticToolsParser
from langchain.utils.openai_functions import convert_pydantic_to_openai_tool
from langchain_core.runnables import Runnable
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.language_models import BaseLanguageModel

_EXTRACTION_TEMPLATE = """提取并保存以下段落中提到的相关实体及其属性。

如果某个属性不存在且在函数参数中不是必需的，请不要将其包含在输出中。"""  # noqa: E501


def create_extraction_chain_pydantic(
    pydantic_schemas: Union[List[Type[BaseModel]], Type[BaseModel]],
    llm: BaseLanguageModel,
    system_message: str = _EXTRACTION_TEMPLATE,
) -> Runnable:
    if not isinstance(pydantic_schemas, list):
        pydantic_schemas = [pydantic_schemas]
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_message),
        ("user", "{input}")
    ])
    tools = [convert_pydantic_to_openai_tool(p) for p in pydantic_schemas]
    model = llm.bind(tools=tools)
    chain = prompt | model | PydanticToolsParser(tools=pydantic_schemas)
    return chain
```